In [4]:
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains

# I used Firefox; you can use whichever browser you like.
browser = webdriver.Chrome()


# Tell Selenium to get the URL you're interested in.
browser.get("https://www.foxnews.com/category/us/environment/climate-change")

# Close Alert Page
alert = browser.find_element_by_css_selector('.alert-banner .close')
alert.click()
time.sleep(3)


while True:
    try:
        button = browser.find_element_by_link_text('Show More')
        ActionChains(browser).move_to_element(button).perform()
        button.click()
        time.sleep(4)
    except Exception as e:
        print (e)
        break

# Now that the page is fully scrolled, grab the source code.
source_data = browser.page_source

# Throw your source into BeautifulSoup and start parsing!
bs_data = BeautifulSoup(source_data, 'html.parser')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".alert-banner .close"}
  (Session info: chrome=76.0.3809.132)


In [15]:
article_list = bs_data.find('div', {'class':'article-list'})
links=[]
# find all a tags
for a in article_list.findAll('a'):
    if a.parent.name == 'h4'and 'video.foxnews' not in a['href']:
        links.append('https://www.foxnews.com'+ a['href'])

In [17]:
len(links)

276

In [34]:
import requests
import pandas as pd

titles=[]
articles=[]

for link in links:
    article=''
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # scrap title if needed
    '''
    title = soup.find('h1', {'class':'headline'})
    titles.append(title.text)
    '''
    
    # scrap article text
    article_container = soup.find('div', {'class':'article-body'})
    if article_container:
        paragraphs = article_container.findAll('p')
        if paragraphs:
            for p in paragraphs:
                if "CLICK HERE" not in p.text:
                    article = article + "" + p.text.replace('\xa0', ' ')
            articles.append(article)

In [30]:
len(articles)

274

In [35]:
articles[5]

'A recent poll ranks climate change and the environment second only to healthcare among the most important issues to Democratic voters in the first-in-the-nation primary state. The hot-button topic is already a main pitch among progressive Democratic candidates flocking to the state ahead of its February primary and residents say the state’s tourism industry is directly impacted by ongoing environmental changes.GREENFIELD, N.H. – The 2020 Democratic presidential hopefuls flocking to the first-in-the-nation primary state of New Hampshire could encounter voters who say they are feeling the effects of climate change in their pocketbooks -- as they seek to woo Democrats who count the environment as a top issue.Tourism is the Granite State’s second-largest industry and accounts for a large portion of the state’s economy. The ski industry alone contributes on average about half a billion dollars a year to the state, but it’s been forced to make some changes over the years to combat a rise in

In [26]:
for article in articles:
    article = article.replace('\xa0', ' ')

In [27]:
articles[5]

'\nA recent poll ranks climate change and the environment second only to healthcare among the most important issues to Democratic voters in the first-in-the-nation primary state. The hot-button topic is already a main pitch among progressive Democratic candidates flocking to the state ahead of its February primary and residents say the state’s tourism industry is directly impacted by ongoing environmental changes.\nGREENFIELD, N.H. –\xa0The 2020 Democratic presidential hopefuls flocking to the first-in-the-nation primary state of New Hampshire\xa0could encounter\xa0voters who say they are feeling the effects of climate change\xa0in their pocketbooks -- as they seek to woo Democrats who count the environment as a top issue.\nTourism is the Granite State’s second-largest industry and accounts for a large portion of the state’s economy. The ski industry alone contributes on average about half a billion dollars a year to the state, but it’s been forced to make some changes over the years t

In [36]:
df = pd.DataFrame(articles, columns=['article']).drop_duplicates()
df['denial?'] = [1]*len(df.index)
#save df to csv
df.to_csv('../data/right/data_fox_articles.csv', index=False)

In [33]:
df

,article,denial?
0,\nCandidates scramble to qualify for president...,1
1,\nFox News Flash top headlines for August 24 a...,1
2,\nFox News Flash top headlines for August 24 a...,1
3,\nBrazil's president has rejected calls from E...,1
4,\nDaniel Turner calls Bernie Sanders' climate ...,1
5,\nA recent poll ranks climate change and the e...,1
6,\nSinger Elton John reveals he paid for a rece...,1
7,\nFox News Flash top headlines for August 18 a...,1
8,\n2020 Democratic presidential contender Sen. ...,1
9,\nEnergy Secretary Rick Perry responds to Demo...,1
